In [2]:
import grpc

# add a folder to the system path
import sys
sys.path.append("../protos")
import pipeline_pb2 as opencv_pb2
import pipeline_pb2_grpc as opencv_pb2_grpc
from aux import wrap_value, unwrap_value
import json
import numpy as np
import io

def bytes_to_np(b: bytes) -> np.ndarray:
    return np.load(io.BytesIO(b))


In [70]:
#target='printart.isr.ist.utl.pt:8061'
target='localhost:8061'
# List of file paths (example paths — replace with your actual ones)
file_paths = [
    '00.jpg',
    '01.jpg',
    '01.jpg',
]

#list all files in a certain directory
import os
directory = "/home/manuelf/mast3r/images_in/piv"
#file_paths = [os.path.join(directory, f) for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))][0:1]

# List to hold the binary data of each image
image_byte_list = []
for path in file_paths:
    # Open the file in binary read mode ('rb') and read its entire content
    with open(path, 'rb') as f:
        image_bytes = f.read()
        image_byte_list.append(image_bytes)
        print(f"Read {path}: {len(image_bytes) / (1024 * 1024):.2f} MB")

in_json = {
    "opencv": {
        "parameters": {
            "feature_extractor": "SIFT"
        }
    }
}

request = opencv_pb2.Envelope(config_json = json.dumps(in_json),
                              data = {"images": wrap_value(image_byte_list)})

#Para imagens muito grandes buffer grande
channel_opt = [('grpc.max_send_message_length',-1), 
               ('grp.max_receive_message_length',-1),
               ('grpc.max_message_length', -1)]

channel=grpc.insecure_channel(target,options=channel_opt)
estimator_stub = opencv_pb2_grpc.PipelineServiceStub(channel)

Read 00.jpg: 0.16 MB
Read 01.jpg: 0.16 MB
Read 01.jpg: 0.16 MB


In [ ]:
response = estimator_stub.Process(request)

kpts = bytes_to_np(unwrap_value(response.data["keypoints"]))
#descs = bytes_to_np(unwrap_value(response.data["descriptors"]))
try:
    inliersA = bytes_to_np(unwrap_value(response.data["matches_inliers_a"]))
    inliersB = bytes_to_np(unwrap_value(response.data["matches_inliers_b"]))

    inliersA.shape == inliersB.shape
except:
    print("No inliers")

print(response.config_json)

No inliers
{"status": "success", "runtime": 0.24432802200317383, "timestamp": 1759749625.7711535}


I0000 00:00:1759749633.625198   39960 chttp2_transport.cc:1336] ipv6:%5B::1%5D:8061: Got goaway [2] err=UNAVAILABLE:GOAWAY received; Error code: 2; Debug Text: Cancelling all calls {grpc_status:14, http2_error:2}
